In [17]:
import pymc3 as pm
import theano
import theano.tensor as tt
import numpy as np
import pandas as pd
import arviz as az
import subprocess

func_dict = {"mean": np.mean, 
             "q2.5": lambda x: np.percentile(x, 2.5), 
             "q97.5": lambda x: np.percentile(x, 97.5)}

output_dir = "../../results/sens_t0"
# !rm -rf {output_dir}
!mkdir -p {output_dir}
output_data_dir = output_dir + "/datasets"
!mkdir -p {output_data_dir}


from scipy.integrate import quad

class Integrate(theano.Op):
    def __init__(self, expr, var, *extra_vars):
        super().__init__()
        self._expr = expr
        self._var = var
        self._extra_vars = extra_vars
        self._func = theano.function(
            [var] + list(extra_vars),
            self._expr,
            on_unused_input='ignore')
    
    def make_node(self, start, stop, *extra_vars):
        self._extra_vars_node = extra_vars
        assert len(self._extra_vars) == len(extra_vars)
        self._start = start
        self._stop = stop
        vars = [start, stop] + list(extra_vars)
        return theano.Apply(self, vars, [tt.dscalar().type()])
    
    def perform(self, node, inputs, out):
        start, stop, *args = inputs
        val = quad(self._func, start, stop, args=tuple(args))[0]
        out[0][0] = np.array(val)
        
    def grad(self, inputs, grads):
        start, stop, *args = inputs
        out, = grads
        replace = dict(zip(self._extra_vars, args))
        
        replace_ = replace.copy()
        replace_[self._var] = start
        dstart = out * theano.clone(-self._expr, replace=replace_)
        
        replace_ = replace.copy()
        replace_[self._var] = stop
        dstop = out * theano.clone(self._expr, replace=replace_)

        grads = tt.grad(self._expr, self._extra_vars)
        dargs = []
        for grad in grads:
            integrate = Integrate(grad, self._var, *self._extra_vars)
            darg = out * integrate(start, stop, *args)
            dargs.append(darg)
            
        return [dstart, dstop] + dargs

In [19]:
%%time

!rm -rf data_tmp
!mkdir -p data_tmp

start = theano.shared(0.)
stop = theano.shared(250.)
μ = theano.shared(2.838)
σ = theano.shared(0.520)

for idx, day0 in enumerate(range(10)[6:]):
    t0 = '2019-12-%02d'%(day0+1)
    CUTOFF_TIME = '2020-01-24'
    
    print(t0)
    subprocess.call(['Rscript', 'prepare_data.R', './data_tmp', t0, CUTOFF_TIME])
    
    df = pd.read_csv("data_tmp/data.csv")
    df_onset2death = pd.read_csv("data_tmp/data_onset2death.csv")
    df_onset2report = pd.read_csv("data_tmp/data_onset2report.csv")

    for idx0, flnm in enumerate(['data.csv', 'data_onset2death.csv', 'data_onset2report.csv']):
        if idx0==0:
            !cp data_tmp/{flnm} {output_data_dir}/{t0}_{flnm}
        else:
            !cp data_tmp/{flnm} {output_data_dir}/{flnm}
            
    if (idx==0):
        # module for onset2report
        with pm.Model() as model_reporting_delay:
            a_delay = pm.HalfNormal('a_delay', sd=5)
            b_delay = pm.HalfCauchy('b_delay', 2.5)
            timeOnsetToReport = df_onset2report.dist.values
            pm.Gamma('likelihood_delay', a_delay, b_delay, observed=timeOnsetToReport)
            pm.Deterministic('mean_delay', a_delay/b_delay);
            pm.Deterministic('sd_delay', np.sqrt(a_delay)/b_delay);
            trace_reporting_delay = pm.sample(10000, tune=5000, cores=10, target_accept=.85, init='advi')

        res_delay = pm.summary(trace_reporting_delay, var_names=['a_delay', 'b_delay', 'mean_delay'])['mean']
        df_res = az.summary(trace_reporting_delay, var_names=['mean_delay', 'sd_delay', 'a_delay', 'b_delay'], stat_funcs=func_dict, extend=False, round_to=5).reset_index().rename(columns={'index': 'var'})
        df_res.rename(columns={'q2.5': 'lower', 'q97.5': 'upper'}).loc[:,['var','mean','lower','upper']].\
            to_csv(output_dir+'/onset2report.csv', index=False)

    # main module
    inci_idx = np.min(df.loc[lambda d: d.exports>0].index)
    inci_tmin = df.loc[inci_idx,'time']
    len_p = len(df.loc[lambda d: d['time']>=inci_tmin,'prob_travel'])
    death_idx = np.min(df.loc[lambda d: d['deaths']>0].index)
    with pm.Model() as model:  
        ## main data and priors ##
        K = df['exports'].shape[0]
        exported_cases = df['exports'].values
        p = df.loc[0,'prob_travel']

        neglogr = pm.HalfNormal('neglogr', testval=-np.log(0.1))
        r = pm.Deterministic('r',np.exp(-neglogr))
        T0 = df['time'].values[inci_idx]
        i0 = 1.0

        t = tt.arange(1,K+1,1)
        Incidence = pm.Deterministic('Incidence',i0*(tt.exp(r*t)-1.0)/r)

        ## implementing numerical integration 
        s = tt.dscalar('s')
        s.tag.test_value = np.zeros(()) #variable of integration
        r_ = tt.dscalar('r_')
        r_.tag.test_value = np.ones(())*0.14
        func = tt.exp(-r_*s)/s/σ/((2.0*np.pi)**0.5)*tt.exp(-((tt.log(s)-μ)**2)/2/(σ**2))
        integrate = Integrate(func, s, r_)

        ## calculating us ##
        u_delay = pm.Deterministic('u_delay', (1 + r*res_delay['mean_delay']/res_delay['a_delay'])**(-res_delay['a_delay']))
        u_death = pm.Deterministic('u_death', integrate(start, stop, r))
        ##############################

        ## reconstructed incidence from exportation events ##
        mu = (u_delay*Incidence*p/(1-p))[inci_idx:K]
        alpha = (1.0/(1-p))
        pm.Gamma('likelihood_incidence', mu, alpha, shape=K-death_idx, observed=exported_cases[inci_idx:K])
        ##############################

        ## CFR ##
        death = df['deaths'].values
        neglogq = pm.Gamma('neglogq', 2, .5, shape=K-death_idx, testval=-np.log(.06))
        q = pm.Deterministic('q',np.exp(-neglogq))

        shape_death = u_death*Incidence[death_idx:K]*q/(1-q)
        invscale_death = 1.0/(1-q)
        pm.Gamma('likelihood_death', shape_death, invscale_death, observed=death[death_idx:K])
        ##############################

        pm.Deterministic('predictedDeath', u_death*Incidence[death_idx:K]*q)

        sample = pm.sample(3750, cores=8, tune=2500, target_accept=.92, init='advi')

    df_res = az.summary(sample, 
                        var_names=['r', 'Incidence', 'q', 'u_delay', 'predictedDeath'], 
                        stat_funcs=func_dict, extend=False, round_to=6).reset_index().rename(columns={'index': 'var'})
    df_res['time'] = df_res['var'].apply(lambda st: st[st.find("[")+1:st.find("]")])
    df_res['time'] = ['NA' if "[" not in y else int(x)+1 for x,y in zip(df_res['time'],df_res['var'])]
    df_res['var'] = df_res['var'].apply(lambda st: st[:st.find("[")] if "[" in st else st)
    df_res.loc[lambda d: d['var']=='q', 'var'] = 'CFR'
    df_res.rename(columns={'q2.5': 'lower', 'q97.5': 'upper'}).loc[:,['var','time','mean','lower','upper']].\
        to_csv(output_dir+'/'+t0+'_incidence.csv', index=False)
    
!rm -rf data_tmp

2019-12-07


Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average Loss = 164.62:   5%|▌         | 10891/200000 [00:02<00:47, 3953.90it/s]
Convergence achieved at 11000
Interrupted at 10,999 [5%]: Average Loss = 279.23
Multiprocess sampling (10 chains in 10 jobs)
NUTS: [b_delay, a_delay]
Sampling 10 chains, 0 divergences: 100%|██████████| 150000/150000 [00:33<00:00, 4432.15draws/s]
The number of effective samples is smaller than 25% for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average Loss = 799.07:   6%|▌         | 11389/200000 [01:24<23:25, 134.18it/s]   
Convergence achieved at 11400
Interrupted at 11,399 [5%]: Average Loss = 7.7253e+10
Multiprocess sampling (8 chains in 8 jobs)
NUTS: [neglogq, neglogr]
Sampling 8 chains, 0 divergences: 100%|██████████| 50000/50000 [49:40<00:00, 16.77draws/s] 


2019-12-08


Auto-assigning NUTS sampler...
Initializing NUTS using advi...


KeyboardInterrupt: 